In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from random import choice
pd.options.mode.chained_assignment = None 
import pickle

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Insert-Generic-Name-Here/'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
# from lonelyboy.geospatial import group_patterns as gsgp


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

In [ ]:
PLT_FIG_WIDTH = 13.487
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

## Vessel-Static Data Statistics

In [ ]:
vessel_static = pd.read_csv('./data/csv/nari_static.csv')
vessel_cat_list = pd.read_csv('./data/csv/Ship Types List.csv')
vessel_det_cat_list = pd.read_csv('./data/csv/Ship Types Detailed List.csv')

In [ ]:
print (f'Before: {len(vessel_static)}')
vessel_static = vessel_static.dropna(subset=['shiptype'])
vessel_static = vessel_static.drop_duplicates(subset=['sourcemmsi', 'shiptype'], keep='first')
print (f'After: {len(vessel_static)}')

In [ ]:
vessel_static.head(10)

In [ ]:
len(vessel_static.sourcemmsi.unique())

In [ ]:
for mmsi, mmsi_static in vessel_static.groupby('sourcemmsi'):
    ves_types = [vessel_cat_list.loc[(shiptypeID >= vessel_cat_list.shiptype_min) & (shiptypeID <= vessel_cat_list.shiptype_max)][['type_name', 'ais_type_summary']]['type_name'].values.tolist() for shiptypeID in mmsi_static.shiptype.unique()]
    if (len(ves_types) > 1):
        print (f'Vessel: {mmsi}\t Shiptype(s): {ves_types}')

In [ ]:
vessel_cat_list

In [ ]:
vessel_det_cat_list.head()

In [ ]:
vessel_categories = vessel_cat_list.merge(vessel_det_cat_list, on='id_shiptype', how='inner')
vessel_categories

In [ ]:
vessel_categories.type_name.unique()

In [ ]:
tmp = vessel_static.shiptype.value_counts().to_frame().reset_index()
tmp.columns = ['shiptype', '#records']

tmp['type_name'] = np.nan
tmp['ais_type_summary'] = np.nan

for row in tmp.itertuples(index=True):
    ves_type = vessel_cat_list.loc[(row.shiptype >= vessel_cat_list.shiptype_min) & (row.shiptype <= vessel_cat_list.shiptype_max)][['type_name', 'ais_type_summary']]
    if (len(ves_type) > 0):
        tmp.loc[row.Index, 'type_name'] = ves_type.type_name.values[0]
        tmp.loc[row.Index, 'ais_type_summary'] = ves_type.ais_type_summary.values[0]

tmp.fillna('Unknown', inplace=True)

In [ ]:
tmp

In [ ]:
tmp['ais_type_summary'] = tmp['ais_type_summary'].replace(to_replace=r'[ \t$]{1}', value='\n', regex=True)

In [ ]:
# tmp.groupby('type_name')['#records'].sum().sort_values().to_frame().reset_index().plot.barh(x='type_name', y='#records', cmap='tab20', rot=25, legend=False)
tmp.groupby('ais_type_summary')['#records'].sum().sort_values().to_frame().reset_index().plot.bar(x='ais_type_summary', y='#records', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), title='Vessel Type Popularity', cmap='tab20', rot=0, width=0.95, legend=False)
# plt.ylabel('Vessel Type')
plt.xlabel('')
plt.ylabel('#Vessel-Types')

In [ ]:
tmp.loc[tmp['ais_type_summary'] == 'Cargo']['type_name']

In [ ]:
vessel_static_on_steroids = vessel_static
vessel_static_on_steroids['type_name'] = np.nan
vessel_static_on_steroids['ais_type_summary'] = np.nan

for row in vessel_static_on_steroids.itertuples(index=True):
    ves_type = vessel_cat_list.loc[(row.shiptype >= vessel_cat_list.shiptype_min) & (row.shiptype <= vessel_cat_list.shiptype_max)][['type_name', 'ais_type_summary']]
    if (len(ves_type) > 0):
        vessel_static_on_steroids.loc[row.Index, 'type_name'] = ves_type.type_name.values[0]
        vessel_static_on_steroids.loc[row.Index, 'ais_type_summary'] = ves_type.ais_type_summary.values[0]

vessel_static_on_steroids.fillna('Unknown', inplace=True)

In [ ]:
vessel_static_on_steroids

# Intersecting with the Dynamic Data

In [ ]:
mean_records_per_day = pd.read_csv('./data/csv/stats/mean_records_per_day.csv')
mean_records_per_day.head()

In [ ]:
tmp2 = pd.merge(mean_records_per_day, vessel_static_on_steroids, left_on='mmsi', right_on='sourcemmsi', how='inner', )
tmp2.head()

In [ ]:
tmp2 = tmp2.groupby(['type_name', 'ais_type_summary']).size().sort_values(0, ascending=True).reset_index()
tmp2.columns = ['type_name', 'ais_type_summary', '#vessels']
tmp2

In [ ]:
len(tmp2.loc[tmp2.type_name == 'Fishing'])

In [ ]:
tmp2['ais_type_summary'] = tmp2['ais_type_summary'].replace(to_replace=r'[ \t$]{1}', value='\n', regex=True)
tmp2.groupby('ais_type_summary')['#vessels'].sum().sort_values().to_frame().reset_index().plot.bar(x='ais_type_summary', y='#vessels', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), title='Vessel Type Popularity', cmap='tab20', rot=0, width=0.95, legend=False)
# plt.ylabel('Vessel Type')
plt.xlabel('')
plt.ylabel('#Vessel-Types')

In [ ]:
tmp2['#vessels'].sum()

## Getting Information about Fishing Vessels

In [ ]:
mmsis = vessel_static.loc[vessel_static.shiptype == 30].sourcemmsi.unique()
np.save('fisheries_mmsis.npy', mmsis)

In [ ]:
mmsis = vessel_static.loc[vessel_static.shiptype == 30].sourcemmsi.unique()

import configparser
import psycopg2

properties = configparser.ConfigParser()
properties.read(os.path.join('.','sql_server.ini'))
properties = properties['SERVER']

host    = properties['host']
db_name = properties['db_name']
uname   = properties['uname']
pw      = properties['pw']
port    = properties['port']
con     = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port=port)
cur     = con.cursor()

In [ ]:
traj_sql = 'SELECT * FROM ais_data.dynamic_ships_segmented_12h_resampled_1min WHERE mmsi IN %s'
fishing_vessels_trips = pd.read_sql_query(traj_sql%(tuple(mmsis),), con=con)

In [ ]:
fishing_vessels_trips.columns = ['id', 'mmsi', 'turn', 'speed', 'velocity', 'course', 'heading', 'lon',
                                 'lat', 'ts', 'traj_id', 'datetime', 'real_point',
                                 'trip_id']

In [ ]:
fishing_vessels_trips.to_csv('fisheries_dynamic.csv', header=True, index=False)

In [ ]:
fishing_vessels_trips = pd.read_csv('data/csv/fisheries_dynamic.csv')

In [ ]:
fishing_vessels_trips.loc[fishing_vessels_trips.real_point == 1].to_csv('fisheries_dynamic_real_points.csv', header=True, index=False) 

In [ ]:
# tmp = fishing_vessels_trips.loc[fishing_vessels_trips.mmsi==227650230]
# tmp = gspp.gdf_from_df(tmp)
# gsplt.map_plot(tmp, column='trip_id', color=[None], cmap='viridis')